In [ ]:
!pip install neuralforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.4/249.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT, NHITS
from neuralforecast.losses.pytorch import MAE

In [ ]:
hist_exog_list = ['holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays', 'year', 'month', 'day','day_of_week', 'week_of_year', 'shops_closed_lag','Shop_opened_after_close']

tft_model = TFT(h=60,  # Forecasting horizon of 60 days
                input_size=120,  # Assuming we use 12 months of history
                hist_exog_list=hist_exog_list,
                max_steps=200,
                start_padding_enabled=True,
                loss=MAE())

# Initialize the NHITS model
nhits_model = NHITS(h=60,  # Forecasting horizon of 60 days
                input_size=120,  # Assuming we use 12 months of history
                hist_exog_list=hist_exog_list,
                max_steps=200,
                start_padding_enabled=True,
                loss=MAE())

INFO:lightning_fabric.utilities.seed:Seed set to 1
/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
train_df = pd.read_csv('/content/Train_df_NueralForecast.csv')
X1_df = pd.read_csv('/content/Future_df_NueralForecast.csv')

In [ ]:
train_df.drop('Unnamed: 0', axis=1, inplace=True)
X1_df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
train_df['ds'] = pd.to_datetime(train_df['ds'])
X1_df['ds'] = pd.to_datetime(X1_df['ds'])

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4528 entries, 0 to 4527
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   unique_id                4528 non-null   object        
 1   ds                       4528 non-null   datetime64[ns]
 2   y                        4528 non-null   float64       
 3   holiday                  4528 non-null   int64         
 4   shops_closed             4528 non-null   int64         
 5   winter_school_holidays   4528 non-null   int64         
 6   school_holidays          4528 non-null   int64         
 7   year                     4528 non-null   int64         
 8   month                    4528 non-null   int64         
 9   day                      4528 non-null   int64         
 10  day_of_week              4528 non-null   int64         
 11  week_of_year             4528 non-null   int64         
 12  shops_closed_lag         4528 non-

In [ ]:
nf = NeuralForecast(models=[tft_model, nhits_model], freq='D')  # 'D' for Daily frequency

nf.fit(df=train_df)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MAE                      | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 3.3 K  | train
4 | static_encoder          | StaticCovariateEncoder   | 331 K  | train
5 | temporal_encoder        | TemporalCovariateEncoder | 1.5 M  | train
6 | temporal_fusion_decoder | TemporalFusi

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=200` reached.


In [ ]:
ft_fcst = nf.predict(futr_df = X1_df)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:209: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [ ]:
ft_fcst

,ds,TFT,NHITS
unique_id,,,
Brno_1,2024-01-16,8848.027344,8985.985352
Brno_1,2024-01-17,8882.442383,9230.323242
Brno_1,2024-01-18,8889.479492,9203.248047
Brno_1,2024-01-19,8889.146484,9239.326172
Brno_1,2024-01-20,8886.718750,9335.618164
...,...,...,...
Prague_3,2024-03-11,5340.330078,5619.524414
Prague_3,2024-03-12,5340.690430,5754.593262
Prague_3,2024-03-13,5340.901367,5818.606934


In [ ]:
ft_fcst.to_csv("Rohlik_daily_nueral.csv")